In [1]:
# get testcase
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)

simple_queries = pd.read_csv("simpleQueries.csv", sep=";", encoding="utf-8")
complex_queries = pd.read_csv("complexQueries.csv", sep=",", encoding="cp1252")

data_file = pd.read_csv("v15.3.1.csv", sep=",", encoding="utf-8")
data_file = data_file[['entry_id', 'title']]

In [2]:
simple_queries.head()

,Question,Expected
0,How to use Route Group?,"v15_3_1_08-route-groups_convention,v15_3_1_11-parallel-routes_examples,v15_3_1_invalid-href-passed_useful links,v15_3_1_invalid-relative-url-external-as_useful links,v15_3_1_react-hydration-error_possible ways to fix it"
1,How to generate static params in Dynamic Routes?,"v15_3_1_10-dynamic-routes_example,v15_3_1_10-dynamic-routes_generating static params,v15_3_1_react-hydration-error_possible ways to fix it,v15_3_1_16-progressive-web-apps_creating a pwa with next_js,v15_3_1_index_apis"
2,How to use Parallel Routes?,"v15_3_1_react-hydration-error_possible ways to fix it,v15_3_1_16-progressive-web-apps_creating a pwa with next_js,v15_3_1_index_apis"
3,"What is the naming convention for Intercepting Routes, show me some examples.","v15_3_1_12-intercepting-routes_convention,v15_3_1_next-prerender-sync-request_useful links,v15_3_1_invalid-i18n-config_useful links,v15_3_1_react-hydration-error_possible ways to fix it"
4,What is Middleware and when does it run?,"v15_3_1_react-hydration-error_possible ways to fix it,v15_3_1_16-progressive-web-apps_creating a pwa with next_js,v15_3_1_07-amp_amp validation,v15_3_1_index_apis,v15_3_1_image_advanced props"


In [3]:
complex_queries.head()

,Question,Expected
0,"How can I implement a dynamic post editing feature in a Next.js application using the App Router?\n\nThe post editing form is a Client Component utilizing useState for local state management.?\n\nThe form submits data to a Server Action defined with the ""use server"" directive.?\n\nUpon successful submission, the application revalidates the cache for the updated post and redirects the user to the post's detail page.?\n\nThe application provides feedback to the user during the submission process, such as loading indicators and error messages.?\n\nAdditionally, how can I ensure that sensitive operations, like updating the post in the database, are securely handled on the server side, and that the client does not have direct access to sensitive information??\n\nPlease provide code snippets in TypeScript that demonstrate this functionality.","v15_3_1_react-hydration-error_possible ways to fix it,v15_3_1_07-amp_amp validation,v15_3_1_10-dynamic-routes_convention,v15_3_1_10-dynamic-routes_example,v15_3_1_10-dynamic-routes_typescript"
1,"How can I implement a responsive image gallery in a Next.js application using the App Router?\n\nImages are stored in the public directory and served as static assets.?\n\nThe gallery utilizes the next/image component to optimize images for different device sizes and resolutions.?\n\nThe application ensures that images are lazy-loaded and do not cause layout shifts during loading.?\n\nThe gallery supports both local images from the public directory and remote images from a trusted external source, with appropriate configuration in next.config.js.?\n\nThe application provides a fallback mechanism for images that fail to load.?\n\nAdditionally, how can I configure caching headers for the static assets served from the public directory to improve performance, considering that Next.js applies a default Cache-Control: public, max-age=0 header?","v15_3_1_react-hydration-error_possible ways to fix it,v15_3_1_07-amp_amp validation,v15_3_1_04-images-and-fonts_optimizing images,v15_3_1_01-images_image sizing,v15_3_1_image_responsive images"
2,"You're building a dashboard-style internal tool using the Next.js App Router, where performance is critical. You want to:\n\nLazy load heavy Client Components (e.g., charts or data visualizations) that arenï¿½t immediately needed on first render.\n\nEnsure type-safety using TypeScript for components and props.\n\nUse dynamic imports with suspense: true and provide a custom loading fallback UI.\n\nOnly load third-party libraries like react-chartjs-2 on the client side to reduce bundle size.\n\nHow would you set this up properly across your layout, pages, and components? Include code snippets for the following:\n1. components/Chart.tsx (Client Component with chart)\n```\n'use client'\n\nimport { Chart as ChartJS, ArcElement, Tooltip, Legend } from 'chart.js';\nimport { Doughnut } from 'react-chartjs-2';\n\nChartJS.register(ArcElement, Tooltip, Legend);\n\ntype ChartProps = {\n title: string;\n data: number[];\n};\n\nexport default function Chart({ title, data }: ChartProps) {\n return (\n <div className=""p-4"">\n <h2>{title}</h2>\n <Doughnut\n data={{\n labels: ['A', 'B', 'C'],\n datasets: [{\n label: title,\n data,\n backgroundColor: ['#F00', '#0F0', '#00F'],\n }]\n }}\n />\n </div>\n );\n}\n```\n\ncomponents/ChartLazy.tsx\n```\nimport dynamic from 'next/dynamic';\n\nconst Chart = dynamic(() => import('./Chart'), {\n ssr: false,\n loading: () => <p>Loading chart...</p>,\n suspense: true,\n});\n\nexport default Chart;\n```\n\napp/dashboard/page.tsx\n```\nimport { Suspense } from 'react';\nimport Chart from '@/components/ChartLazy';\n\nexport default function DashboardPage() {\n return (\n <main className=""space-y-8"">\n <h1>Dashboard</h1>\n <Suspense fallback={<div>Loading Charts...</div>}>\n <Chart title=""User Stats"" data={[10, 20, 30]} />\n </Suspense>\n </main>\n );\n}\n```","v15_3_1_react-hydration-error_possible ways to fix it,v15_3_1_07-amp_

In [4]:
data_file.head()

,entry_id,title
0,v15_3_1_index_what is next_js?,index - Welcome to the Next.js Documentation. - What is Next.js?
1,v15_3_1_index_main features,index - Welcome to the Next.js Documentation. - Main Features
2,v15_3_1_index_how to use these docs,index - Welcome to the Next.js Documentation. - How to Use These Docs
3,v15_3_1_index_app router vs pages router,index - Welcome to the Next.js Documentation. - App Router vs Pages Router
4,v15_3_1_index_pre-requisite knowledge,index - Welcome to the Next.js Documentation. - Pre-Requisite Knowledge


In [5]:
import requests
import json

test_api = "http://localhost:8000/prompt/generate"


In [6]:
def precision_at_k(k: int, relevant_docs: list = [], retrieved_docs: list = []):
    return len(set(relevant_docs) & set(retrieved_docs)) / k

def recall_at_k(k: int, relevant_docs: list = [], retrieved_docs: list = []):
    return len(set(relevant_docs) & set(retrieved_docs)) / len(set(relevant_docs))

def get_titles(entry_ids: list = []):
    docs = data_file[data_file["entry_id"].isin(entry_ids)].to_dict("records")
    # print(docs)
    return docs


In [ ]:
# test simple queries
simple_queries_overall = pd.DataFrame(columns=["K", "Precision@K", "Recall@K", "Latency-Retriever", "Latency-Generator"])

for at_k in [1,3,5,7]:
    body = {
        "version_name": "v15.3.1",
        "query": "",
        "model": "llama3.2:3b",
        "additional_options": {
            "retriever_options": {
            "top_k": at_k},
            "generator_options": {}
        }
    }
    print(f"@{at_k}")
    simple_queries_results = pd.DataFrame(columns=["Precision", "Recall", "Latency-Retriever", "Latency-Generator"])
    save_file_name = f"simple_queries_results_at{at_k}.csv"
    for index, row in simple_queries.iterrows():
        question = row["Question"]
        ids = row["Expected"].split(",")
        expected_titles = [data['title'] for data in get_titles(ids)]
        body["query"] = question
        response = requests.post(test_api, json=body)
        # get_titles(ids)
        result = json.loads(response.text)
        retrieved_docs = result['retrieved_data']['docs']
        # print("=======")
        # print("Expected:",expected_titles)
        # print("Got",retrieved_docs)
        precision = precision_at_k(at_k, expected_titles, retrieved_docs)
        recall = recall_at_k(at_k, expected_titles, retrieved_docs)
        print(f"Precision: {precision}, Recall: {recall}")
        latency_ret = result['latency']['retrieve_time']
        latency_gen = result['latency']['generate_time']
        simple_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=simple_queries_results.columns), simple_queries_results], ignore_index=True)
    simple_queries_results.to_csv(os.path.join('results', save_file_name), sep=',', encoding='utf-8')
    precision_atk = simple_queries_results['Precision'].mean()
    recall_atk = simple_queries_results['Recall'].mean()
    lat_ret = simple_queries_results['Latency-Retriever'].mean()
    lat_gen = simple_queries_results['Latency-Generator'].mean()
    simple_queries_overall = pd.concat([pd.DataFrame([[at_k ,precision_atk, recall_atk, lat_ret, lat_gen]], columns=simple_queries_overall.columns), simple_queries_overall], ignore_index=True)
# result = json.loads(response.text)
# print(result)
simple_queries_overall.to_csv("simple_queries_overall.csv", sep=',', encoding='utf-8')

@1
Precision: 0.0, Recall: 0.0


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\2594947956.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  simple_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=simple_queries_results.columns), simple_queries_results], ignore_index=True)


Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 1.0, Recall: 0.2
Precision: 1.0, Recall: 0.2
Precision: 1.0, Recall: 0.2
Precision: 1.0, Recall: 0.2
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
@3


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\2594947956.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  simple_queries_overall = pd.concat([pd.DataFrame([[at_k ,precision_atk, recall_atk, lat_ret, lat_gen]], columns=simple_queries_overall.columns), simple_queries_overall], ignore_index=True)


Precision: 0.3333333333333333, Recall: 0.2


...

Precision: 0.3333333333333333, Recall: 0.2
Precision: 0.3333333333333333, Recall: 0.3333333333333333
Precision: 0.3333333333333333, Recall: 0.25
Precision: 1.0, Recall: 0.6
Precision: 1.0, Recall: 0.6
Precision: 1.0, Recall: 0.6
Precision: 1.0, Recall: 0.6
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.3333333333333333, Recall: 0.3333333333333333
Precision: 0.3333333333333333, Recall: 0.25
Precision: 0.3333333333333333, Recall: 0.5
Precision: 0.3333333333333333, Recall: 1.0
Precision: 0.0, Recall: 0.0
@5
Precision: 0.8, Recall: 0.8


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\2594947956.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  simple_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=simple_queries_results.columns), simple_queries_results], ignore_index=True)


Precision: 0.6, Recall: 0.6
Precision: 0.6, Recall: 1.0
Precision: 0.6, Recall: 0.75
Precision: 1.0, Recall: 1.0
Precision: 1.0, Recall: 1.0
Precision: 1.0, Recall: 1.0
Precision: 1.0, Recall: 1.0
Precision: 0.2, Recall: 0.5
Precision: 0.2, Recall: 0.5
Precision: 0.6, Recall: 1.0
Precision: 0.4, Recall: 0.5
Precision: 0.2, Recall: 0.5
Precision: 0.2, Recall: 1.0
Precision: 0.2, Recall: 1.0
@7
Precision: 0.42857142857142855, Recall: 0.6


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\2594947956.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  simple_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=simple_queries_results.columns), simple_queries_results], ignore_index=True)


Precision: 0.42857142857142855, Recall: 0.6
Precision: 0.2857142857142857, Recall: 0.6666666666666666
Precision: 0.2857142857142857, Recall: 0.5
Precision: 0.5714285714285714, Recall: 0.8
Precision: 0.7142857142857143, Recall: 1.0
Precision: 0.7142857142857143, Recall: 1.0
Precision: 0.7142857142857143, Recall: 1.0
Precision: 0.14285714285714285, Recall: 0.5
Precision: 0.14285714285714285, Recall: 0.5
Precision: 0.42857142857142855, Recall: 1.0
Precision: 0.2857142857142857, Recall: 0.5
Precision: 0.14285714285714285, Recall: 0.5
Precision: 0.14285714285714285, Recall: 1.0
Precision: 0.14285714285714285, Recall: 1.0


In [10]:
# test complex queries
complex_queries_overall = pd.DataFrame(columns=["K", "Precision@K", "Recall@K", "Latency-Retriever", "Latency-Generator"])

for at_k in [1,3,5,7]:
    body = {
        "version_name": "v15.3.1",
        "query": "",
        "model": "llama3.2:3b",
        "additional_options": {
            "retriever_options": {
            "top_k": at_k},
            "generator_options": {}
        }
    }
    print(f"@{at_k}")
    complex_queries_results = pd.DataFrame(columns=["Precision", "Recall", "Latency-Retriever", "Latency-Generator"])
    save_file_name = f"complex_queries_results_at{at_k}.csv"
    for index, row in complex_queries.iterrows():
        question = row["Question"]
        ids = row["Expected"].split(",")
        expected_titles = [data['title'] for data in get_titles(ids)]
        body["query"] = question
        response = requests.post(test_api, json=body)
        # get_titles(ids)
        result = json.loads(response.text)
        retrieved_docs = result['retrieved_data']['docs']
        # print("=======")
        # print("Expected:",expected_titles)
        # print("Got",retrieved_docs)
        precision = precision_at_k(at_k, expected_titles, retrieved_docs)
        recall = recall_at_k(at_k, expected_titles, retrieved_docs)
        print(f"Precision: {precision}, Recall: {recall}")
        latency_ret = result['latency']['retrieve_time']
        latency_gen = result['latency']['generate_time']
        complex_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=complex_queries_results.columns), complex_queries_results], ignore_index=True)
    complex_queries_results.to_csv(os.path.join('results', save_file_name), sep=',', encoding='utf-8')
    precision_atk = complex_queries_results['Precision'].mean()
    recall_atk = complex_queries_results['Recall'].mean()
    lat_ret = complex_queries_results['Latency-Retriever'].mean()
    lat_gen = complex_queries_results['Latency-Generator'].mean()
    complex_queries_overall = pd.concat([pd.DataFrame([[at_k ,precision_atk, recall_atk, lat_ret, lat_gen]], columns=complex_queries_overall.columns), complex_queries_overall], ignore_index=True)
# result = json.loads(response.text)
# print(result)
complex_queries_overall.to_csv("complex_queries_overall.csv", sep=',', encoding='utf-8')

@1
Precision: 0.0, Recall: 0.0


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\3661239888.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complex_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=complex_queries_results.columns), complex_queries_results], ignore_index=True)


Precision: 0.0, Recall: 0.0
Precision: 1.0, Recall: 0.25
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
@3


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\3661239888.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complex_queries_overall = pd.concat([pd.DataFrame([[at_k ,precision_atk, recall_atk, lat_ret, lat_gen]], columns=complex_queries_overall.columns), complex_queries_overall], ignore_index=True)


Precision: 0.3333333333333333, Recall: 0.2


...

Precision: 0.3333333333333333, Recall: 0.2
Precision: 0.0, Recall: 0.0
Precision: 0.3333333333333333, Recall: 0.25
Precision: 0.3333333333333333, Recall: 0.2
Precision: 0.0, Recall: 0.0
Precision: 0.3333333333333333, Recall: 0.08333333333333333
Precision: 0.3333333333333333, Recall: 0.125
Precision: 0.3333333333333333, Recall: 0.14285714285714285
Precision: 0.3333333333333333, Recall: 0.125
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.3333333333333333, Recall: 0.2
Precision: 0.3333333333333333, Recall: 0.125
Precision: 0.3333333333333333, Recall: 0.2
@5
Precision: 0.4, Recall: 0.4


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\3661239888.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complex_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=complex_queries_results.columns), complex_queries_results], ignore_index=True)


Precision: 0.4, Recall: 0.4
Precision: 0.0, Recall: 0.0
Precision: 0.4, Recall: 0.5
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.4, Recall: 0.16666666666666666
Precision: 0.4, Recall: 0.25
Precision: 0.4, Recall: 0.2857142857142857
Precision: 0.4, Recall: 0.25
Precision: 0.2, Recall: 0.2
Precision: 0.2, Recall: 0.14285714285714285
Precision: 0.4, Recall: 0.4
Precision: 0.4, Recall: 0.25
Precision: 0.4, Recall: 0.4
@7
Precision: 0.2857142857142857, Recall: 0.4


C:\Users\PC\AppData\Local\Temp\ipykernel_3732\3661239888.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complex_queries_results = pd.concat([pd.DataFrame([[precision, recall, latency_ret, latency_gen]], columns=complex_queries_results.columns), complex_queries_results], ignore_index=True)


Precision: 0.2857142857142857, Recall: 0.4
Precision: 0.0, Recall: 0.0
Precision: 0.2857142857142857, Recall: 0.5
Precision: 0.0, Recall: 0.0
Precision: 0.0, Recall: 0.0
Precision: 0.2857142857142857, Recall: 0.16666666666666666
Precision: 0.2857142857142857, Recall: 0.25
Precision: 0.2857142857142857, Recall: 0.2857142857142857
Precision: 0.2857142857142857, Recall: 0.25
Precision: 0.14285714285714285, Recall: 0.2
Precision: 0.14285714285714285, Recall: 0.14285714285714285
Precision: 0.2857142857142857, Recall: 0.4
Precision: 0.2857142857142857, Recall: 0.25
Precision: 0.2857142857142857, Recall: 0.4


In [11]:
# precision_mean = complex_queries_results['Precision'].mean()
# recall_mean = complex_queries_results['Recall'].mean()
# latency_ret_mean = complex_queries_results['Latency-Retriever'].mean()
# latency_gen_mean = complex_queries_results['Latency-Generator'].mean()

# print(f"Precision: {precision_mean}, Recall: {recall_mean}, Latency-Retriever: {latency_ret_mean}, Latency-Generator: {latency_gen_mean}")